In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
306,"I watched Love Life on holiday, when it was fi...",positive
16,It is so nice to see Bruce Willis come down of...,positive
310,"Why, o' WHY! ...did I pick this one up? Well.....",negative
644,I've watched a lot of television in my 51 year...,positive
561,This movie was incredible. I would recommend i...,positive


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [9]:
import nltk 
nltk.download('stopwords')
nltk.download('wordnet')
df = normalize_text(df)
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,review,sentiment
483,movie start bunch dead men walking peep sittin...,positive
109,bonnie clyde thelma louise fine road movie set...,positive
766,movie fun board br br essentially follows come...,positive
35,ridiculous comedy given arm flailing direction...,negative
951,usually big fan pacino scarface serpico devil ...,negative


In [5]:
df['sentiment'].value_counts()

sentiment
negative    261
positive    239
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
306,"I watched Love Life on holiday, when it was fi...",1
16,It is so nice to see Bruce Willis come down of...,1
310,"Why, o' WHY! ...did I pick this one up? Well.....",0
644,I've watched a lot of television in my 51 year...,1
561,This movie was incredible. I would recommend i...,1


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [14]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
import dagshub
mlflow.set_tracking_uri('https://dagshub.com/naveensddp1/rahil-capstone-project.mlflow')
dagshub.init(repo_owner='naveensddp1', repo_name='rahil-capstone-project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

# import mlflow
# with mlflow.start_run():
# mlflow.log_param('parameter name', 'value')
# mlflow.log_metric('metric name', 1)
  

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\Admin\anaconda3\envs\atlas\lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=7c69de3f-e5e3-4a02-a96d-ca88c215cf97&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=c37e3a982259b2010b440b56980efa1bf7e28a2f06527fc071a5ab020d5fe6cf




Accessing as naveensddp1

Initialized MLflow to track repo "naveensddp1/rahil-capstone-project"

Repository naveensddp1/rahil-capstone-project initialized!

2025/12/27 15:09:19 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/2baa6497461d441db41a94d2f16d150d', creation_time=1766828359387, experiment_id='0', last_update_time=1766828359387, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [16]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-12-27 15:19:00,757 - INFO - Starting MLflow run...
2025-12-27 15:19:01,712 - INFO - Logging preprocessing parameters...
2025-12-27 15:19:02,873 - INFO - Initializing Logistic Regression model...
2025-12-27 15:19:02,874 - INFO - Fitting the model...
2025-12-27 15:19:02,919 - INFO - Model training complete.
2025-12-27 15:19:02,920 - INFO - Logging model parameters...
2025-12-27 15:19:03,444 - INFO - Making predictions...
2025-12-27 15:19:03,446 - INFO - Calculating evaluation metrics...
2025-12-27 15:19:03,464 - INFO - Logging evaluation metrics...
2025-12-27 15:19:04,872 - INFO - Saving and logging the model...
2025/12/27 15:19:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-12-27 15:19:16,823 - INFO - Model training and logging completed in 15.11 seconds.
2025-12-27 15:19:16,824 - INFO - Accuracy: 0.7
2025-12-27 15:19:16,825 - INFO - Precision: 0.6415094339622641
2025-12-27 15:19:16,825 - INFO - Recall: 0.7555555555555555
2025-12-27 1

🏃 View run zealous-swan-869 at: https://dagshub.com/naveensddp1/rahil-capstone-project.mlflow/#/experiments/0/runs/a3858b66f7394374aa3e664b8e9f5e5a
🧪 View experiment at: https://dagshub.com/naveensddp1/rahil-capstone-project.mlflow/#/experiments/0
